In [17]:
import pandas as pd
import numpy as np
import requests
import time
import os,sys,os.path

In [21]:

API_XAPP_TOKEN = os.environ.get('API_XAPP_TOKEN')

In [2]:
df = pd.read_csv('/home/mollyppl/code/molpl/artsy-fartsci/data/artists.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           50 non-null     int64 
 1   name         50 non-null     object
 2   years        50 non-null     object
 3   genre        50 non-null     object
 4   nationality  50 non-null     object
 5   bio          50 non-null     object
 6   wikipedia    50 non-null     object
 7   paintings    50 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 3.2+ KB


In [4]:
df['birth_year'] = df.years.apply(lambda x: int(x.replace('–','-').split(' - ')[0]))
df['death_year'] = df.years.apply(lambda x: int(x.replace('–','-').split(' - ')[1]))

In [5]:
old_art_df = df[df['death_year'] < 1900]

In [6]:
modern_art_df = df[df['death_year'] > 1900]

In [7]:
modern_art_df.head()

,id,name,years,genre,nationality,bio,wikipedia,paintings,birth_year,death_year
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193,1884,1920
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88,1866,1944
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70,1886,1957
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73,1840,1926
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194,1898,1967


In [8]:
cols = ['artwork_id',
            'title',
            'category',
            'medium',
            'date',
            'height_cm',
            'width_cm',
            'image_url_template',
            'collecting_institution',
            'image_url_normalized']

In [9]:
artworks_df = pd.DataFrame(columns=cols)

In [11]:
modern_art_df.iloc[0,:]

id                                                             0
name                                           Amedeo Modigliani
years                                                1884 - 1920
genre                                              Expressionism
nationality                                              Italian
bio            Amedeo Clemente Modigliani (Italian pronunciat...
wikipedia         http://en.wikipedia.org/wiki/Amedeo_Modigliani
paintings                                                    193
birth_year                                                  1884
death_year                                                  1920
Name: 0, dtype: object

In [31]:
# get info into format to fit with the rest of the data - artists
artist_dict = {'id':[],
               'slug':[],
               'name':[],
               'gender':[],
               'birthday':[],
               'deathday':[],
               'hometown':[],
               'location':[],
               'nationality':[]
               }
for ix, row in modern_art_df.iterrows():
    artist_dict["id"].append('')
    artist_dict["slug"].append(row['name'].lower().replace(' ','-')) # name-like-this
    artist_dict["name"].append(row["name"])
    if row['name'] == 'Frida Kahlo':
        gender = 'female'
    else:
        gender = 'male'
    artist_dict["gender"].append(gender) # add this manually - but i think they're all men 
    artist_dict["birthday"].append(row['birth_year'])
    artist_dict["deathday"].append(row['death_year'])
    artist_dict["hometown"].append('') # google
    artist_dict["location"].append('') # where they worked
    artist_dict["nationality"].append(row['nationality'])

In [32]:
artist_formatted_df = pd.DataFrame(artist_dict)

In [41]:
artworks_artsy_df = pd.read_csv('bq_data.csv')

In [47]:
modern_art_df

,id,name,years,genre,nationality,bio,wikipedia,paintings,birth_year,death_year
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193,1884,1920
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88,1866,1944
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70,1886,1957
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73,1840,1926
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194,1898,1967
5,5,Salvador Dali,1904 - 1989,Surrealism,Spanish,Salvador Domingo Felipe Jacinto Dalí i Domènec...,http://en.wikipedia.org/wiki/Salvador_Dalí,139,1904,1989
9,9,Gustav Klimt,1862 - 1918,"Symbolism,Art Nouveau",Austrian,"Gustav Klimt (July 14, 1862 – February 6, 1918...",http://en.wikipedia.org/wiki/Gustav_Klimt,117,1862,1918
11,11,Kazimir Malevich,1879 - 1935,Suprematism,Russian,Kazimir Severinovich Malevich (February 23 [O....,http://en.wikipedia.org/wiki/Kazimir_Malevich,126,1879,1935
12,12,Mikhail Vrubel,1856 - 1910,Symbolism,Russian,Mikhail Aleksandrovich Vrubel (Russian: Михаи́...,http://en.wikipedia.org/wiki/Mikhail_Vrubel,171,1856,1910
13,13,Pablo Picasso,1881 - 1973,Cubism,Spanish,Pablo Ruiz Picasso (; Spanish: [ˈpaβlo piˈkaso...,http://en.wikipedia.org/wiki/Pablo_Picasso,439,1881,1973


In [42]:
artworks_artsy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6052 entries, 0 to 6051
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   artwork_id              6052 non-null   object 
 1   title                   6052 non-null   object 
 2   category                6052 non-null   object 
 3   medium                  6042 non-null   object 
 4   date                    5013 non-null   object 
 5   height_cm               4530 non-null   float64
 6   width_cm                4515 non-null   float64
 7   image_url_template      6052 non-null   object 
 8   collecting_institution  5884 non-null   object 
 9   image_url_normalized    6052 non-null   object 
dtypes: float64(2), object(8)
memory usage: 472.9+ KB


In [53]:
# get artwork info
# generate x rows where x = number of paintings for that artist
# how to link it to final embedding??
# also where to store image for front end - can it be in its own url? the kaggle url?
artworks_all = artworks_artsy_df.copy()
for ix, row in modern_art_df.iterrows():
    for i in range(row['paintings']):
        temp = {}
        temp['artwork_id'] = ''
        temp['title'] = f"{row['name']} {i}"
        temp['category'] = 'Painting'
        temp['medium'] = None
        temp['date'] = None
        temp['height_cm'] = None
        temp['width_cm'] = None
        temp['image_url_template'] = None
        temp['collecting_institution'] = None
        temp['image_url_normalized'] = None
        temp_df = pd.DataFrame(temp, index=[0])
        artworks_all = pd.concat([artworks_all,temp_df],ignore_index=True)



In [54]:
artworks_all

,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
0,4eb1c899c8004a000100deb3,Portrait of a Young Woman with a White Coif,Painting,Oil and tempera on panel,1541,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/liVgLUFu...,NaN,https://d32dm0rphc51dk.cloudfront.net/liVgLUFu...
1,4f99da873314020001000700,The Alba Madonna,Painting,Oil on panel transferred to canvas,ca. 1510,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/lnnzsg3v...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/lnnzsg3v...
2,515b1d46056351dc33001014,The Fall of Man [middle panel],Painting,Oil on hardboard transferred from panel,ca. 1535,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/kBOdyvyl...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/kBOdyvyl...
3,515ce23e7b7057eb4c00115c,The Rule of Mars [right panel],Painting,Oil on hardboard transferred from panel,ca. 1535,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/hidKPTZj...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/hidKPTZj...
4,515d6a15b5907b33b1004797,Madonna and Child with Saint Mary Magdalene an...,Painting,Tempera on panel transferred to canvas,ca. 1330/1340,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/-SeTFEk1...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/-SeTFEk1...
...,...,...,...,...,...,...,...,...,...,...
6073,,Piet Mondrian 0,Painting,None,None,NaN,NaN,None,None,None
6074,,Joan Miro 0,Painting,None,None,NaN,NaN,None,None,None
6075,,Andy Warhol 0,Painting,None,None,NaN,NaN,None,None,None
6076,,Paul Gauguin 0,Painting,None,None,NaN,NaN,None,None,None
